In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as layer
from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Model
from tensorflow.keras import Model
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold

In [2]:
train = pd.read_csv('./data/criteo_sampled_data.csv')
train.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [3]:
# train.info()

In [4]:
cols = train.columns[1:]

In [5]:
dense_feats = [f for f in cols if f[0] == 'I']
sparse_feats = [f for f in cols if f[0] == 'C']

In [6]:
def process_dense_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna(0)
    for f in feats:
        d[f] = d[f].apply(lambda x: np.log(x+1) if x>-1 else -1)
    return d
data_dense = process_dense_feats(train, dense_feats)
    

In [7]:
def process_sparse_feats(data, feats):
    d = data.copy()
    d = d[feats].fillna('-1')
    for f in feats:
        d[f] = LabelEncoder().fit_transform(d[f])
    return d
data_sparse = process_sparse_feats(train, sparse_feats)

In [8]:
total_data = pd.concat([data_dense, data_sparse], axis=1)
total_data['label'] = train['label']

In [9]:
# 如果你只是想对流经该层的数据做个变换，而这个变换本身没有什么需要学习的参数，那么直接用Lambda Layer是最合适的了

In [10]:
# 获取类别型特征的大小
sparse_feat_config= dict()
for col in sparse_feats:
    sparse_feat_config[col] = total_data[col].nunique()

In [11]:
# 构造验证集和训练集
train_data = total_data.loc[:500000-1]
valid_data = total_data.loc[500000:]

train_dense_x = [train_data[f].values for f in dense_feats]#  train_data[dense_feats] 
train_sparse_x = [train_data[f].values for f in sparse_feats] # train_data[sparse_feats] # 
train_label = train_data['label'].values
train_label = tf.cast(train_label, tf.int32)

val_dense_x = [valid_data[f].values for f in dense_feats] # valid_data[dense_feats]   
val_sparse_x = [valid_data[f].values for f in sparse_feats] # valid_data[sparse_feats]
val_label = valid_data['label'].values
val_label = tf.cast(val_label, tf.int32)


In [12]:
# 构造训练集和测试集
def make_data(total_data,idx):
    train_data = total_data.loc[idx,:]
    train_dense_x = [train_data[f].values for f in dense_feats]
    train_sparse_x = [train_data[f].values for f in sparse_feats]
    train_label = train_data['label'].values
    return train_sparse_x,train_dense_x,train_label

# 写法一
继承layer,定义不同功能的层

In [13]:
# 独立层：嵌入层
class Embedding_dense(tf.keras.layers.Layer):
    def __init__(self,sparse_feat_config, embeding_shape):
        super(Embedding_dense, self).__init__()
        # l2正则化
        self.reg_1 = tf.keras.regularizers.l2(0.1)
        self.embed_first = {}
        self.sparse_feat_config = sparse_feat_config
        self.embeding_shape = embeding_shape
        self.sparse_feat = list(sparse_feat_config.keys())
        for key, value in self.sparse_feat_config.items():
            self.embed_first[key] = layer.Embedding(value+1,self.embeding_shape, 
                                                    embeddings_regularizer=self.reg_1, 
                                                    name='embed'+key)
    def call(self,x_sparse):
        embed_lookup_first = []
        for i,key in enumerate(self.sparse_feat):

            _embed = self.embed_first[key](x_sparse[i])

            embed_lookup_first.append(_embed)

        return embed_lookup_first
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'sparse_feat_config': self.sparse_feat_config,
            'embeding_shape': self.embeding_shape,
        })
        return config
# t = Embedding_dense(sparse_feat_config,1)
# y = t(inputs_sparse)    

In [14]:
# 独立层：DNN
class DNN(tf.keras.layers.Layer):
    def __init__(self,dnn_config=[128,128,64]):
        super(DNN, self).__init__()
        self.dnn_layers_config = dnn_config
        self.dnn_layers = []
        for s in self.dnn_layers_config:
            self.dnn_layers.append(layer.Dense(s))
            
    def call(self,embed_lookup_second):
#         embed_lookup_second = layer.Concatenate(axis=1)(embed_lookup_second) 
        fc_layer = layer.Flatten()(embed_lookup_second)
        for i,_ in enumerate(self.dnn_layers):
            fc_layer = self.dnn_layers[i](fc_layer)
        
        return fc_layer
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'dnn_layers_config': self.dnn_layers_config,
        })
        return config

In [18]:
# 交叉层
class cin_cross_layer(tf.keras.layers.Layer):
    def __init__(self,x_0):
        super(cin_cross_layer,self).__init__()
        self.x_0 = x_0
        
    def build(self, input_shape):
        self.w = tf.Variable(
                    initial_value = tf.random_normal_initializer()(shape=(input_shape[-1],1),dtype='float32'),
                    trainable=True
                 )
        self.b = tf.Variable(tf.zeros(shape=(1,input_shape[-1])),trainable=True)
    def call(self, x_l):
        # x_0 *x_l^T
        
        t1 = tf.expand_dims(self.x_0,axis=1)
        t2 = tf.expand_dims(x_l,axis=1)
        x_0l = tf.matmul(t1,t2, transpose_a = True)
        x_lw = tf.matmul(x_0l,self.w)
        x_lw = tf.squeeze(x_lw,axis=-1)
        return x_lw +  x_l + self.b
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'x_0': self.x_0,
        })
        return config  

In [21]:
class DCN:
    def __init__(self,sparse_feat_config,dense_feats):
        
        self.sparse_feat_config= sparse_feat_config
        self.inputs_sparse, self.inputs_dense = self.build_input(sparse_feat_config,dense_feats)
        
#         self.firsr_cross_dense = firsr_cross_dense(self.sparse_feat_config)
        self.Embedding_dense = Embedding_dense(self.sparse_feat_config,8)
#         self.second_cross_dense = second_cross_dense()
        self.DNN = DNN([128,128,64])
        
        self.DCN =  self.build_model() 
    def build_input(self,sparse_feat_config,dense_feats):
        inputs_sparse = []
        inputs_dense = []
        for key in sparse_feat_config:
            inputs_sparse.append(layer.Input(shape=(1,),name=key))
        for key in dense_feats:
            inputs_dense.append(layer.Input(shape=(1,),name=key))
        
        return inputs_sparse, inputs_dense

    def build_model(self, num_lays = 3):
        # DCN输入部分
        # sparse特征嵌入
        sparse_embed_lookup =  self.Embedding_dense(self.inputs_sparse) 
        concat_sparse_inputs = layer.Concatenate(axis=1)(sparse_embed_lookup)
        concat_sparse_inputs = layer.Flatten()(concat_sparse_inputs)
        # dense 特征
        concat_dense_inputs = layer.Concatenate(axis=1)(self.inputs_dense)
        # 拼接最终的输入特征
        emded_inputs = layer.Concatenate(axis=1)([concat_sparse_inputs,concat_dense_inputs])
        
        # Cross
        cross_layers = []
        for i in range(num_lays):
            cross_layers.append(cin_cross_layer(emded_inputs))
        cross_layer = cin_cross_layer(emded_inputs)
        x_l = emded_inputs
        for i in range(num_lays):
            x_l = cross_layers[i](x_l)
#         print('cross',x_l)
        
        # DNN部分
        fc_layer = self.DNN(emded_inputs)
#         print(fc_layer)

        # 组合
        stack_layer = layer.Concatenate()([x_l, fc_layer])
        output_layer = layer.Dense(1,activation = 'sigmoid',use_bias=True)(stack_layer)
#         print('output_layer',output_layer)
        # 初始化模型
        model = Model(self.inputs_sparse + self.inputs_dense, outputs=output_layer)
        model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-3),
                      loss= 'binary_crossentropy',
                      metrics=['AUC'])
        return model
    

    def train(self,train_data,train_label,valid_data, valid_label,batch_size,epochs,callbacks):
        self.DCN.fit(train_data,train_label,
                  batch_size=batch_size, epochs=epochs, verbose=1, 
                  validation_data=(valid_data, valid_label),
                  callbacks = callbacks
                 )
# DeepFM(sparse_feat_config, dense_feats).build_model()

In [ ]:
# 五折交叉 + 提前停止 + 保存模型
tf.compat.v1.disable_eager_execution()

skf = StratifiedKFold(n_splits = 5, random_state=1996, shuffle = True)
for idx, (train_idx, val_idx) in enumerate(skf.split(total_data,total_data['label'])):
    print('fold:',idx)
    K.clear_session()
    train_sparse_x,train_dense_x,train_label = make_data(total_data,train_idx)
    val_sparse_x,val_dense_x,val_label = make_data(total_data,val_idx) 
    # 定义回调
    
    # 保存模型
    file_path = f'./model/{idx}.h5'

    checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True, mode='min')
    # metric 不提高时，减小学习率
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=1, min_lr=0.0001, verbose=1)
    # val_loss 连续两次提升小于 1e-2，提前停止
    earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2,verbose=1, mode='auto')
    callbacks = [checkpoint, reduce_lr, earlystopping]

    # 初始化模型
    DCN_ctr = DCN(sparse_feat_config, dense_feats)
    DCN_ctr.train(train_sparse_x+train_dense_x,train_label,
                 val_sparse_x+val_dense_x,val_label,
                128,1, callbacks=callbacks)    

fold: 0
Train on 480000 samples, validate on 120000 samples
479872/480000 [============================>.] - ETA: 0s - loss: 2.1758 - AUC: 0.7281
Epoch 00001: val_loss improved from inf to 0.70290, saving model to ./model/0.h5
480000/480000 [==============================] - 116s 242us/sample - loss: 2.1754 - AUC: 0.7281 - val_loss: 0.7029 - val_AUC: 0.7404 - lr: 0.0010
fold: 1
Train on 480000 samples, validate on 120000 samples
479872/480000 [============================>.] - ETA: 0s - loss: 2.1778 - AUC: 0.7269
Epoch 00001: val_loss improved from inf to 0.69248, saving model to ./model/1.h5
480000/480000 [==============================] - 115s 239us/sample - loss: 2.1774 - AUC: 0.7269 - val_loss: 0.6925 - val_AUC: 0.7447 - lr: 0.0010
fold: 2
Train on 480000 samples, validate on 120000 samples
479872/480000 [============================>.] - ETA: 0s - loss: 2.1767 - AUC: 0.7275
Epoch 00001: val_loss improved from inf to 0.69297, saving model to ./model/2.h5
480000/480000 [============

In [ ]:
# 模型的加载与预测
DCN_ctr = DCN(sparse_feat_config, dense_feats).build_model()
DCN_ctr.load_weights('./model/0.h5')
DCN_ctr.predict(val_sparse_x+val_dense_x)